In [20]:
import pandas as pd

## Datasets & Cleaning

### Battlefield

In [21]:
# Loading raw datasets
bf4_data = pd.read_csv("../assets/raw/SteamDB Battlefield 4.csv")
bfh_data = pd.read_csv("../assets/raw/SteamDB Battlefield Hardline.csv")
bf1_data = pd.read_csv("../assets/raw/SteamDB Battlefield 1.csv")
bfv_data = pd.read_csv("../assets/raw/SteamDB Battlefield V.csv")
bf2042_data = pd.read_csv("../assets/raw/SteamDB Battlefield 2042.csv")

In [ ]:
fields_to_keep = ['month', 'peak', 'gain', '% gain'] # Fields we want to keep
list_of_bf_data = [bf4_data, bfh_data, bf1_data, bfv_data, bf2042_data] w

# Cleaning the datasets in list_of_bf_data
for i in range(len(list_of_bf_data)):
    df = list_of_bf_data[i]
    df.columns = df.columns.str.lower()
    df = df.replace("-", "0")
    df[['peak', 'gain', '% gain']] = df[['peak', 'gain', '% gain']].replace({",": "", "%": ""}, regex=True).astype(float)
    df = df[fields_to_keep][1:] # Getting rid of first row of Last 30 days
    df["month"] = pd.to_datetime(df["month"], format="%b-%y")
    df = df.sort_values("month")
    list_of_bf_data[i] = df

In [23]:
# Merging the cleaned datasets
bf_merged = pd.concat(list_of_bf_data, ignore_index=True)
bf_merged = bf_merged[fields_to_keep]

In [24]:
# Grouping by month and summing up the peak players and gain
SteamDB_bf_merged_grouped = bf_merged.groupby(bf_merged.columns[0]).sum()

SteamDB_bf_merged_grouped = SteamDB_bf_merged_grouped.reset_index()
SteamDB_bf_merged_grouped = SteamDB_bf_merged_grouped.sort_values("month")

# Recalculate % gain
SteamDB_bf_merged_grouped['% gain'] = ((SteamDB_bf_merged_grouped['peak']-SteamDB_bf_merged_grouped['peak'].shift(1))*100/SteamDB_bf_merged_grouped['peak'].shift(1)).round(1)
SteamDB_bf_merged_grouped.iloc[0,3] = 0 # Setting the first % gain to 0

In [25]:
# Exporting cleaned dataset
SteamDB_bf_merged_grouped.to_csv("../assets/clean/SteamDB_Battlefield_Clean.csv", encoding="utf-8", index=False, header=True)